<table style="width:100%; border-collapse: collapse;">
  <tr>
    <td style="width:20%; vertical-align:middle;">
      <img src="LogoUVG.png" width="400"/>
    </td>
    <td style="text-align:left; vertical-align:middle;">
      <h2 style="margin-bottom: 0;">Universidad del Valle de Guatemala - UVG</h2>
      <h3 style="margin-top: 0;">Facultad de Ingeniería - Computación</h3>
      <p style="font-size: 16px; margin-bottom: 0; margin-top: -20px">
        <strong>Curso:</strong> CC3104 - Aprendizaje por Refuerzo 
        <strong>Sección:</strong> 10
      </p>
      <p style="font-size: 16px; margin: 0;"><strong>Laboratorio 5:</strong> Time Difference Learning</p>
      <br>
      <p style="font-size: 15px; margin: 0;"><strong>Autores:</strong></p>
      <ul style="margin-top: 5px; padding-left: 20px; font-size: 15px;">
        <li>Diego Alexander Hernández Silvestre - <strong>21270</strong></li>
        <li>Linda Inés Jiménez Vides - <strong>21169</strong></li>
        <li>Mario Antonio Guerra Morales - <strong>21008</strong></li>
      </ul>
    </td>
  </tr>
</table>

## 📝 Task 1

**1. Defina y explique qué “expected sarsa”**
**a. ¿Cómo se diferencia de “sarsa”?**
**b. ¿Para qué sirven las modificaciones que se hacen sobre “sarsa”?**

- Expected SARSA es una variante del algoritmo original en el que, en vez de actualizar el valor Q usando solamente la acción que se tomó en el siguiente estado, este utiliza el valor esperado de todas las acciones posibles de ese estado, siendo ponderado por la probabilidad de que cada una sea elegida según la política actual. Su diferencia con sarsa se da en que, en lugar de tomar Q(s', a'), toma la sumatoria de todos los valores Q de todas las acciones en s'. Su utilidad en las modificaciones se da en reducir la varianza de la estimación por cada iteración, suavizando el aprendizaje.

**2. Defina y explique qué es “n-step TD”**
**a. ¿Cómo se diferencia de TD(0)?**
**b. ¿Cuál es la utilidad de esta modificación?**
**c. ¿Qué usa como objetivo?**

- Es un método de Temporal Difference que actualiza los valores Q utilizando el retorno acumulado de n-pasos en el futuro, en vez de hacerlo solo con uno. La diferencia de TD(0) es que n-step TD utiliza la suma de recompensas de los siguientes n-pasos más el valor estimado en el estado alcanzado. Su utilidad permite un balance entre el sesgo que se puede generar y la varianza, considerando los valores pequeños y grandes de n. El objetivo que usa es el propio retorno de n-step.



**3. ¿Cuál es la diferencia entre SARSA y Q-learning?**

- SARSA es un algoritmo de naturaleza on-policy que aprende el valor Q siguiendo la política mientras esta se está evaluando y mejorando. Mientras que, Q-Learning, por otro lado, es de naturaleza off-policy, y este aprende el valor Q de la política óptima independientemente de la que se siga para explorar.

## 📝 Task 2

En este laboratorio, compararán el rendimiento de SARSA y Q-Learning, dos algoritmos de aprendizaje de refuerzo populares, utilizando el entorno CliffWalking-v0 de la biblioteca Gymnasium. Analizará y graficará las recompensas por episodio y responderá preguntas para profundizar su comprensión de las diferencias entre estos algoritmos.

**Instalar las bibliotecas necesarias**

In [1]:
!pip install matplotlib
!pip install gymnasium==0.28.1


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from gymnasium.envs.toy_text.cliffwalking import CliffWalkingEnv


**Inicialice el entorno**

In [3]:
gym.envs.registration.register(
    id="CliffWalking-v0",
    entry_point="gymnasium.envs.toy_text.cliffwalking:CliffWalkingEnv",
)

env = gym.make("CliffWalking-v0")

n_states = env.observation_space.n
n_actions = env.action_space.n

# Parámetros por defecto
ALPHA = 0.5         # tasa de aprendizaje
GAMMA = 0.99        # factor de descuento
EPSILON = 0.1       # épsilon para política epsilon-greedy
EPISODES = 500      # cantidad de episodios
SEED = 24

# Semillas
np.random.seed(SEED)
env.reset(seed=SEED)

print(f"Entorno de CliffWalking-v0 | Estados: {n_states} | Acciones: {n_actions}")

Entorno de CliffWalking-v0 | Estados: 48 | Acciones: 4


c:\Users\mague\Downloads\Lab5RL\.venv\Lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment CliffWalking-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [4]:
def epsilon_greedy_action(Q, state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.randint(Q.shape[1])
    return np.argmax(Q[state])

def make_epsilon_schedule(epsilon_start, epsilon_end=None, decay_episodes=None):
    if epsilon_end is None or decay_episodes is None or decay_episodes <= 0:
        return lambda ep: epsilon_start
    
    def schedule(ep):
        t = min(ep / decay_episodes, 1.0)
        return (1 - t) * epsilon_start + t * epsilon_end
    return schedule

**Implementar SARSA**

In [5]:
def train_sarsa(env, episodes=EPISODES, alpha=ALPHA, gamma=GAMMA,
                epsilon=EPSILON, epsilon_schedule=None, seed=SEED):
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    Q = np.zeros((n_states, n_actions), dtype=np.float64)
    rewards_per_episode = []

    np.random.seed(seed)

    for ep in range(episodes):
        eps = epsilon_schedule(ep) if epsilon_schedule else epsilon
        state, info = env.reset(seed=seed + ep)
        action = epsilon_greedy_action(Q, state, eps)

        total_reward = 0.0

        for _ in range(max_steps):
            next_state, reward, terminated, truncated, info = env.step(action)
            total_reward += reward

            # Elegir siguiente acción con la misma política
            next_action = epsilon_greedy_action(Q, next_state, eps)

            # Actualización de SARSA
            td_target = reward + gamma * Q[next_state, next_action] * (1 - int(terminated or truncated))
            td_error  = td_target - Q[state, action]
            Q[state, action] += alpha * td_error

            state, action = next_state, next_action

            if terminated or truncated:
                break

        rewards_per_episode.append(total_reward)

    return rewards_per_episode, Q

**Implementar Q-Learning**

**Ejecutar ambos algoritmos**

**Gráfico de los resultados**

**Analice los resultados**

**a. Observe los resultados graficados y compare el rendimiento de SARSA y Q-Learning a lo largo de los episodios.**

- Respuesta

**b. Considere cómo cada algoritmo equilibra la exploración y la explotación.**

- Respuesta


**Responda las siguientes preguntas:**

**a. P1: ¿Qué diferencias observa en los patrones de recompensa entre SARSA y Q-Learning?**

- Respuesta

**b. P2: ¿Por qué Q-Learning tiende a ser más optimista en sus actualizaciones de valor Q en comparación con SARSA?**

- Respuesta

**c. P3: ¿Cómo afecta la naturaleza "on-policy" de SARSA a su proceso de aprendizaje en comparación con la naturaleza "off-policy" de Q-Learning?**

- Respuesta

**d. P4: Según las penalizaciones y recompensas del entorno, ¿qué algoritmo parece aprender el camino más seguro? ¿Por qué?**

- Respuesta

**e. P5: ¿Cómo podría afectar la disminución de épsilon con el tiempo al rendimiento de cada algoritmo?**

- Respuesta


**Preguntas para responder:**

**1. ¿Cuál es el valor estimado de mantener diferentes niveles de existencias para cada producto?**

- Respuesta

**2. ¿Cómo afecta el valor epsilon en la política blanda al rendimiento?**

- Respuesta

**3. ¿Cuál es el impacto de utilizar el aprendizaje fuera de la política en comparación con el aprendizaje dentro de la política?**

- Respuesta
